In [ ]:
import sys
sys.path.append('../htm_rl/htm_rl/')

from agent.agent import Agent, AgentRunner
from agent.memory import Memory, TemporalMemory
from agent.planner import Planner
from common.sa_sdr_encoder import SaSdrEncoder, format_sa_superposition
from common.int_sdr_encoder import IntSdrEncoder, IntRangeEncoder
from common.int_sdr_encoder import SequenceSdrEncoder
from envs.gridwirld_pomdp import GridWorld


import numpy as np
import matplotlib.pyplot as plt

Example of Agent in gridworld environment with two-value state.

In [ ]:
world_description = [[2,0,0],
                     [1,1,0],
                     [0,0,0]]

In [ ]:
gw = GridWorld(world_description, (3, 3), agent_initial_position={'row': 2, 'column': 0})

In [ ]:
gw.world_size

In [ ]:
gw.render()

In [ ]:
gw.observation()

In [ ]:
gw.observable_state

In [ ]:
max_steps = 12

state_encoder = SequenceSdrEncoder('state',
                                   encoders=[IntSdrEncoder('distance',
                                                                   gw.world_size[0],
                                                                   5,
                                                                   4),
                                             IntSdrEncoder('surface', 3, 5, 4),
                                             IntRangeEncoder('row', (-(gw.world_size[0]-1),
                                                                     gw.world_size[1]-1), 5, 4),
                                             IntRangeEncoder('column', (-(gw.world_size[0]-1),
                                                                     gw.world_size[1]-1), 5, 4),
                                             IntSdrEncoder('direction', 4, 5, 4)],
                                   size=5)

In [ ]:
print(state_encoder.value_bits)
state_encoder.total_bits

In [ ]:
action_encoder = IntSdrEncoder('action', gw.n_actions,
                              value_bits=6, activation_threshold=4)

In [ ]:
sa_encoder = SaSdrEncoder(state_encoder, action_encoder)

In [ ]:
sa_encoder.total_bits, sa_encoder.value_bits, sa_encoder.activation_threshold

In [ ]:
action_encoder.activation_threshold

In [ ]:
sparsity = sa_encoder.value_bits / sa_encoder.total_bits
sparsity

In [ ]:
tm = TemporalMemory(n_columns=sa_encoder.total_bits,
                    cells_per_column=16,
                    activation_threshold=sa_encoder.activation_threshold,
                    learning_threshold=sa_encoder.activation_threshold,
                    initial_permanence=0.5,
                    connected_permanence=0.5,
                    maxNewSynapseCount=sa_encoder.value_bits,
                    maxSynapsesPerSegment=sa_encoder.value_bits*2,
                    permanenceIncrement=0.1,
                    permanenceDecrement=0.02,
                    predictedSegmentDecrement=0.001
                    )

In [ ]:
memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition)

In [ ]:
planner = Planner(memory, 10, 1)

In [ ]:
agent = Agent(memory, planner, gw.n_actions)

In [ ]:
run = AgentRunner(agent, gw, 250, max_steps, 0, 1)

In [ ]:
run.run()


In [ ]:
run.agent.planner.inter_episode_goal_memory._set

In [ ]:
run.agent.set_planning_horizon(10)
run.max_steps = max_steps
run.verbosity = 1
run.pretrain = 0
run.n_episodes = 10
run.run()

In [ ]:
fig = plt.figure(figsize=(10, 7))
steps = np.array(run.train_stats.steps[:])
plt.plot(np.arange(steps.size), steps, '.')

In [ ]:
steps.mean(), steps.std()

In [ ]:
#goal_state = (0, 2, -2, 1, 2)
#run.agent.planner.add_goal(goal_state)
run.agent.set_planning_horizon(10)
run.max_steps = max_steps
run.verbosity = 3
run.pretrain = 0
run.n_episodes = 1
run.run()

In [ ]:
import sys

In [ ]:
sys.getsizeof(run